In [0]:
from bs4 import BeautifulSoup
import urllib
import requests
import pandas as pd

#state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
state_names = ["Michigan", "Illinois", "Indiana"]
df = pd.DataFrame(columns = ['state', 'title', 'company', 'salary', 'description', 'url']) 

In [8]:
for state in state_names:
  url = "https://www.indeed.com/jobs?q=data+scientist&l=" + state
  for page_number in range (0,2):
    url = url + "&start=" + str(10*page_number)
    #print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")
    
    for job in soup.find_all("a", class_ = "jobtitle turnstileLink "): 
      job_url  = "https://www.indeed.com" + job['href'] 
      temp_df = pd.DataFrame(columns = ['state', 'title', 'company', 'salary', 'description', 'url']) 
      #print(job_url)
      try:
        job_soup = BeautifulSoup(urllib.request.urlopen(job_url)) 
      except:
        pass
      try:
        job_soup = BeautifulSoup(urllib.request.urlopen(job_url)) 
      except:
        pass
      temp_df = temp_df.append({'url' : url} , ignore_index=True)
      temp_df['state'] = state
      try:
        #print(job_soup.find("h3", attrs={"class":"jobsearch-JobInfoHeader-title"}).text)
        temp_df['title'] = job_soup.find("h3", attrs={"class":"jobsearch-JobInfoHeader-title"}).text
      except: 
        pass
      try:
        #print(job_soup.find("div", attrs={"class":"icl-u-lg-mr--sm icl-u-xs-mr--xs"}).text)
        temp_df['company'] = job_soup.find("div", attrs={"class":"icl-u-lg-mr--sm icl-u-xs-mr--xs"}).text
      except:
        pass
      try:
        #print(job_soup.find("span", attrs={"class":"icl-u-xs-mr--xs"}).text)
        temp_df['salary'] = job_soup.find("span", attrs={"class":"icl-u-xs-mr--xs"}).text
      except:
        pass
      try:
        #print(job_soup.find("div", attrs={"class":"jobsearch-jobDescriptionText"}).text)
        temp_df['description'] = job_soup.find("div", attrs={"class":"jobsearch-jobDescriptionText"}).text
      except:
        pass
      df = df.append(temp_df, ignore_index=True)
      temp_df.drop(axis=0, index=0)
df.head(64)

,state,title,company,salary,description,url
0,Michigan,Machine Learning Engineer,Quicken Loans,NaN,Who We Are\nWe’re America’s largest mortgage l...,https://www.indeed.com/jobs?q=data+scientist&l...
1,Michigan,Data Scientist,DISYS,$65 an hour,Data Scientist/ AI EngineerLocation: Auburn Hi...,https://www.indeed.com/jobs?q=data+scientist&l...
2,Michigan,Data Scientist,clinc,NaN,Who We Are\n\nWe started Clinc in 2015 with th...,https://www.indeed.com/jobs?q=data+scientist&l...
3,Michigan,Sr Data Analyst,Ally Financial Inc.,NaN,As a data analyst for Auto Consumer Credit Ope...,https://www.indeed.com/jobs?q=data+scientist&l...
4,Michigan,Data Scientist 10774 (NO C2C),NaN,NaN,Description:The AI Engineer will be responsibl...,https://www.indeed.com/jobs?q=data+scientist&l...
...,...,...,...,...,...,...
59,Michigan,Data Scientist,Domino's,NaN,"Domino’s Pizza, which began in 1960 as a singl...",https://www.indeed.com/jobs?q=data+scientist&l...
60,Michigan,Data Scientists,Advantage Resourcing,NaN,You will be responsible for providing the data...,https://www.indeed.com/jobs?q=data+scientist&l...
61,Michigan,Sr. Data Scientist,Venteon,"$120,000 a year",Job Description\nPerform Operational Support.\...,https://www.indeed.com/jobs?q=data+scientist&l...
62,Michigan,Data Scientist,DISYS,$65 an hour,Data Scientist/ AI EngineerLocation: Auburn Hi...,https://www.indeed.com/jobs?q=data+scientist&l...


In [9]:
df.shape

(140, 6)